## Задача 1
Нужно написать две программы: Первая генерирует бинарный файл (min 2Гб), состоящий из случайных 32-рязрядных беззнаковых целых чисел (big endian). Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.

Реализуйте две версии:
1. Простое последовательное чтение 
2. Многопоточная + memory-mapped files. 

Сравните время работы.

In [2]:
import numpy as np
import time
import threading

In [3]:
N = 2 ** (1 + 10 + 10 + 10 + 3 - 5)
N

536870912

In [4]:
array = np.random.randint(0, 2 ** 32, N, dtype=np.uint32)
with open("numbers", 'wb') as f:         
    f.write(array.data)

In [5]:
def simple_calculation(array: np.array):
    array_sum = 0
    min_value, max_value = 2 ** 32 + 1, -1
    for i in range(array.shape[0]):
        array_sum += array[i]
        if array[i] > max_value:
            max_value = array[i]
        if array[i] < min_value:
            min_value = array[i]
    return array_sum, min_value, max_value

In [6]:
%%time
stored_array = np.fromfile("numbers", dtype=np.uint32)
simple_calculation(stored_array)

Wall time: 5min 4s


(1152918908967944801, 0, 4294967284)

In [15]:
%%time
stored_array.astype(np.int64).sum(), stored_array.min(), stored_array.max()

Wall time: 2.75 s


(1152918908967944801, 0, 4294967284)

In [17]:
%%time
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor
import mmap
import threading
from tqdm.notebook import tqdm

N = 2 ** (1 + 10 + 10 + 10 + 3 - 5)

array_sum = 0
min_value, max_value = 2 ** 32 + 1, -1
lock_array_sum, lock_min_value, lock_max_value = threading.Lock(), threading.Lock(), threading.Lock()

def calculation_with_thread(filename, arr_length, offset, lock_array_sum, lock_min_value, lock_max_value):
    lc_array_sum = 0
    lc_min_value, lc_max_value = 2 ** 32 + 1, -1
    array = np.memmap(filename, dtype=np.uint32, mode='r+', offset=offset, shape=arr_length, order='C')
    for i in range(array.shape[0]):
        lc_array_sum += array[i]
        if array[i] > lc_max_value:
            lc_max_value = array[i]
        if array[i] < lc_min_value:
            lc_min_value = array[i]
        
    global array_sum
    global min_value
    global max_value
    with lock_array_sum:
        array_sum += lc_array_sum
    with lock_min_value:
        if lc_min_value < min_value:
            min_value = lc_min_value
    with lock_max_value:
        if lc_max_value > max_value:
            max_value = lc_max_value
    return lc_array_sum, lc_min_value, lc_max_value

executor = ThreadPoolExecutor(max_workers=16)
exec_len = N // (2 ** 4)
exec_b_len = exec_len * 4
futures = [executor.submit(calculation_with_thread,
                           "numbers",
                           exec_len,
                           i * exec_b_len,
                           lock_array_sum,
                           lock_min_value,
                           lock_max_value) for i in range(N // exec_len + 1)]

for future in futures:
    print(future.result())

array_sum, min_value, max_value

(72049789882868446, 15, 4294967095)
(72058533373420514, 138, 4294966988)
(72067146033854981, 53, 4294967184)
(72062965186296472, 14, 4294966889)
(72060261058718320, 239, 4294966887)
(72051714255488508, 9, 4294967169)
(72068524803990611, 25, 4294967284)
(72058617598655537, 244, 4294967223)
(72053385254804228, 76, 4294967072)
(72038848056256511, 12, 4294967004)
(72061220215400541, 3, 4294967188)
(72058517408414754, 47, 4294967275)
(72056857990125930, 0, 4294967189)
(72058434082907863, 167, 4294967246)
(72053041806246027, 186, 4294967168)
(72061051960495558, 40, 4294966927)
(0, 0, 0)
Wall time: 14min 12s


(1152918908967944801, 0, 4294967284)